In [1]:
using FileIO, ImageMagick, Colors, FixedPointNumbers, DelimitedFiles, PyPlot, LinearAlgebra, HDF5

### Upload velocity, airfoil position and vorticity contours

In [2]:
data =  "C:/Users/jjser/Desktop/Julia_FTLE_Karthik_Data/airfoil2D_f0.15_amp25deg_KM/data_JJ"
name = "VelData.h"
vel = h5open(joinpath(data,name), "r") do file
    read(file, "A")
end
print("DONE")

DONE

In [3]:
name =  "BvoData.h"
bvo = h5open(joinpath(data,name), "r") do file
    read(file, "A")
end
print("DONE")

DONE

In [4]:
name =  "xR.h"
x_range = h5open(joinpath(data,name), "r") do file
    read(file, "A")
end
name =  "yR.h"
y_range = h5open(joinpath(data,name), "r") do file
    read(file, "A")
end
print("DONE")

DONE

### Upload particle position and FTLE data

In [ ]:
name = "p500.h" 
p500 = h5open(joinpath(data,name), "r") do file
    read(file, "A")
end
name = "n500.h"
n500 = h5open(joinpath(data,name), "r") do file
    read(file, "A")
end
print("DONE")

<div class="alert alert-block alert-success">  
These will have 122 timesteps
</div>

## Plotting figures

### Plot flow map at a certain time step

In [ ]:
function fm(o, t, levels, save=false)
    fig, ax = subplots(figsize=(18.5,10.5))
    cm = ax.contourf(o[1,1,:,:],o[2,1,:,:],o[1,t,:,:], cmap=ColorMap("rainbow"), levels=levels)
    fig.colorbar(cm)
    #title("Flow Map at "*string(move*(t-1)*tstep_delta+t_0) *"ms")
    if save == true
        savefig(joinpath(data, name * string(t) * ".png"), dpi=100)
        close()
    end
end

### Plot FTLE field for both forward and backward time at a certain timestep

In [ ]:
function ftle(p, n, lim, t, save=false)
    x = p[1,1,:,:]
    y = p[2,1,:,:]
    pos = (p[3,t,:,:] .> lim * maximum(p[3,t,:,:])) .|> Int
    neg = (n[3,t,:,:] .> lim * maximum(n[3,t,:,:])) .|> Int
    fig, ax = subplots(figsize=(9.25,5.25))
    levels = range(0, 1,length=3) .+ 0.5
    ax.contourf(x,y,pos,cmap=ColorMap("Blues"),levels=levels, alpha=0.5, zorder=1)
    ax.contourf(x,y,neg,cmap=ColorMap("Reds") ,levels=levels, alpha=0.5, zorder=2)
end

#### 500 by 400 resolution takes around 1 hour

In [ ]:
fm(p500,61,range(9,13,length=50))
fm(n500,61,range(7,11,length=50))

In [ ]:
ftle(p500, n500, 0.8, 122)

### Plot particle flow

In [ ]:
function PlotAll(field, ts, save=false)
    fig, ax = subplots(figsize=(18.5,10.5))
    time = Int(tstep_start+(ts-1)*tstep_delta)
    ax.quiver(x_range[1:end-1],y_range[1:end-1],Vel_contour[1,ts,:,:],Vel_contour[2,ts,:,:], zorder=2)
    levels = range(0, 1,length=3) .+ 0.5
    ax.contourf(x_contour[1:end-1],y_contour[1:end-1],misc_contour[1,ts,:,:], levels=levels,cmap=ColorMap("PuOr"), zorder=1)
    m = maximum(misc_contour[2,1,:,:])
    step = m/30
    levels = range(-1.0,1.0,length=30)
    ax.contourf(x_contour[1:end-1],y_contour[1:end-1],misc_contour[2,ts,:,:],levels=levels,cmap=ColorMap("RdBu_r"),extend="both",zorder=0, alpha=0.5)
    
    ax.scatter(field[1,ts,:,:],field[2,ts,:,:],marker="s",color="pink", zorder=3)
    ax.set_title("This is the original at time " * string(time) * "ms")
    varname = lpad(time,7,'0')
    ax.set_xlim([x_start_loc,x_end_loc])
    ax.set_ylim([y_start_loc,y_end_loc])
    if save==true
        savefig(joinpath(overall_path, save_path, folder, name * "_" * varname * ".png"), dpi=100)
    cat   close()
    end
end